In [1]:
from models.ConvolutionalGenerativeAdversarialNetwork import ConvolutionalGenerativeAdversarialNetwork

Using TensorFlow backend.


In [2]:
image_dim = (28, 28, 1)
latent_dim = (100, )

In [3]:
generator_initial_dim = (7, 7, 8)
generator_activation = 'relu'
discriminator_activation = 'relu'
use_batch_norm = True
generator_convolutional_params = [
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 1, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 2, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 1, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 2, },
    {'strides': (1, 1), 'filters': 1, 'kernel_size': (5, 5), 'upsample': 1, },
    ]
discriminator_convolutional_params = [
    {'strides': (1, 1), 'filters': 8, 'kernel_size': (3, 3),},
    {'strides': (1, 1), 'filters': 8, 'kernel_size': (3, 3),},
    ]

In [4]:
gan = ConvolutionalGenerativeAdversarialNetwork(
    image_dim=image_dim,
    latent_dim=latent_dim,
    generator_initial_dim=generator_initial_dim,
    generator_activation=generator_activation,
    discriminator_activation=discriminator_activation,
    generator_convolutional_params=generator_convolutional_params,
    discriminator_convolutional_params=discriminator_convolutional_params,
    use_batch_norm=use_batch_norm,
    )

W0531 17:21:53.807901 140317086201600 deprecation.py:506] From /home/comadan/.venv/gdl/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
gan.generator_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_input (InputLayer) (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 392)               39592     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 8)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 8)           32        
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 8)           0         
_________________________________________________________________
generator_conv2d_0 (Conv2D)  (None, 7, 7, 32)          6432      
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 32)          0   